In [6]:
#Optional step to delete session if you get an error that multiple sessions are open
# del session

# Initialize Notebook, import libraries and create Snowflake connection

In [1]:
from snowflake.snowpark.session import Session
from snowflake.snowpark.types import IntegerType, FloatType
from snowflake.snowpark.functions import avg, sum, col, udf, call_udf, call_builtin, year, month
import streamlit as st
import pandas as pd
from datetime import date

# scikit-learn (install: pip install -U scikit-learn)
from sklearn.linear_model import LinearRegression

# Session
connection_parameters = {
   "account": "SFSENORTHAMERICA_DEMO222",
   "user": "vinay",
   "password": "Vigyan2020",
   "warehouse": "compute_wh",
   "role": "accountadmin",
   "database": "summit_hol",
   "schema": "public"
}
session = Session.builder.configs(connection_parameters).create()


# test if we have a connection
session.sql("select current_account() acct, current_warehouse() wh, current_database() db, current_schema() schema, current_version() version").show()


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
-----------------------------------------------------------------------------
|"ACCT"                    |"WH"        |"DB"        |"SCHEMA"  |"VERSION"  |
-----------------------------------------------------------------------------
|SFSENORTHAMERICA_DEMO222  |COMPUTE_WH  |SUMMIT_HOL  |PUBLIC    |7.31.2     |
-----------------------------------------------------------------------------



# Query the data

In [2]:
# SQL queries to explore the data

# What financial data is available as a time-series from FRED?
session.sql("SELECT DISTINCT variable_name FROM CYBERSYN_FINANCIAL__ECONOMIC_ESSENTIALS.CYBERSYN.FINANCIAL_FRED_TIMESERIES").show()

# What is the size of all the time-series data?
session.sql("SELECT COUNT(*) FROM CYBERSYN_FINANCIAL__ECONOMIC_ESSENTIALS.CYBERSYN.FINANCIAL_FRED_TIMESERIES").show()

------------------------------------------------------
|"VARIABLE_NAME"                                     |
------------------------------------------------------
|Consumer Loans: Other Consumer Loans, Domestica...  |
|Liabilities and Capital: Liabilities: Deposits:...  |
|Malaysian Ringgit to U.S. Dollar Spot Exchange ...  |
|Total Fed Funds Sold and Securities Purchased U...  |
|Assets: Liquidity and Credit Facilities: Munici...  |
|Liabilities and Capital: Liabilities: Total Lia...  |
|Deposits, All Commercial Banks, Not seasonally ...  |
|Average Hourly Earnings of All Employees, Minin...  |
|Treasury and Agency Securities: Mortgage-Backed...  |
|Liabilities and Capital: Liabilities: Federal R...  |
------------------------------------------------------

--------------
|"COUNT(*)"  |
--------------
|2130586     |
--------------



In [4]:
# Now use Snowpark dataframe
snow_df_pce = (session.table("CYBERSYN_FINANCIAL__ECONOMIC_ESSENTIALS.CYBERSYN.FINANCIAL_FRED_TIMESERIES")
               .filter(col('VARIABLE_NAME') == 'Personal Consumption Expenditures: Chain-type Price Index, Seasonally adjusted, Monthly, Index 2012=100')
               .filter(col('DATE') >= '1972-01-01')
               .filter(month(col('DATE')) == 1))
pd_df_pce_year = snow_df_pce.select(year(col("DATE")).alias('"Year"'), col('VALUE').alias('PCE')).orderBy(col('DATE')).to_pandas()
pd_df_pce_year


AttributeError: 'DataFrame' object has no attribute 'show'

# Train the Linear Regression model

In [ ]:
# train model with PCE index

x = pd_df_pce_year["Year"].to_numpy().reshape(-1,1)
y = pd_df_pce_year["PCE"].to_numpy()

model = LinearRegression().fit(x, y)

# test model for 2022
predictYear = 2022
pce_pred = model.predict([[predictYear]])
# print the last 5 years
print (pd_df_pce_year.tail() )
# run the prediction for 2022
print ('Prediction for '+str(predictYear)+': '+ str(round(pce_pred[0],2)))


### Creating a User Defined Function within Snowflake to do the scoring there

In [63]:
def predict_pce(predictYear: int) -> float:
    return model.predict([[predictYear]])[0].round(2).astype(float)

_ = session.udf.register(predict_pce,
                        return_type=FloatType(),
                        input_type=IntegerType(),
                        packages= ["pandas","scikit-learn"],
                        is_permanent=True, 
                        name="predict_pce_udf", 
                        replace=True,
                        stage_location="@udf_stage")

# Test the trained model by invoking the UDF via a SQL statement

In [ ]:
session.sql("select predict_pce_udf(2022)").show()
